<h1> BUILDING A MOVIE RECCOMENDATION SYSTEM USING THE IMDB5000 DATASET <h1>

<h4> Link to Dataset: https://www.kaggle.com/carolzhangdc/imdb-5000-movie-dataset/code </h4> 

<h2> PART 1: DATA EXPLORATORY ANALYSIS</h2>

In [139]:
import numpy as np #Importing libraries and loading the IMDB 5000 Dataset
import pandas as pd
from google.colab import files
dataset= files.upload()
from sklearn.feature_extraction.text import CountVectorizer  
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

Saving movie_metadata.csv to movie_metadata (2).csv


In [140]:
movies = pd.read_csv("movie_metadata.csv") #Converting the csv file into a pandas dataset
movies.head(10)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens ...,8,143,NaN,0.0,NaN,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0
5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Action|Adventure|Sci-Fi,Daryl Sabara,John Carter,212204,1873,Polly Walker,1.0,alien|american civil war|male nipple|mars|prin...,http://www.imdb.com/title/tt0401729/?ref_=fn_t...,738.0,English,USA,PG-13,263700000.0,2012.0,632.0,6.6,2.35,24000
6,Color,Sam Raimi,392.0,156.0,0.0,4000.0,James Franco,24000.0,336530303.0,Action|Adventure|Romance,J.K. Simmons,Spider-Man 3,383056,46055,Kirsten Dunst,0.0,sandman|spider man|symbiote|venom|villain,http://www.imdb.com/title/tt0413300/?ref_=fn_t...,1902.0,English,USA,PG-13,258000000.0,2007.0,11000.0,6.2,2.35,0
7,Color,Nathan Greno,324.0,100.0,15.0,284.0,Donna Murphy,799.0,200807262.0,Adventure|Animation|Comedy|Family|Fantasy|Musi...,Brad Garrett,Tangled,294810,2036,M.C. Gainey,1.0,17th century|based on fairy tale|disney|flower...,http://www.imdb.com/title/tt0398286/?ref_=fn_t...,387.0,English,USA,PG,260000000.0,2010.0,553.0,7.8,1.85,29000
8,Color,Joss Whedon,635.0,141.0,0.0,19000.0,Robert Downey Jr.,26000.0,458991599.0,Action|Adventure|Sci-Fi,Chris Hemsworth,Avengers: Age of Ultron,462669,92000,Scarlett Johansson,4.0,artificial intelligence|based on comic book|ca...,http://www.imdb.com/title/tt2395427/?ref_=fn_t...,1117.0,English,USA,PG-13,250000000.0,2015.0,21000.0,7.5,2.35,118000
9,Color,David Yates,375.0,153.0,282.0,10000.0,Daniel Radcliffe,25000.0,301956980.0,Adventure|Family|Fantasy|Mystery,Alan Rickman,Harry Potter and the Half-Blood Prince,321795,58753,Rupert Grint,3.0,blood|book|love|potion|professor,http://www.imdb.com/title/tt0417741/?ref_=fn_t...,973.0,English,UK,PG,250000000.0,2009.0,11000.0,7.5,2.35,10000


In [141]:
movies['director_name'].value_counts()   #finds the count of values in that column
#len(movies['director_name'].unique())

Steven Spielberg    26
Woody Allen         22
Clint Eastwood      20
Martin Scorsese     20
Ridley Scott        17
                    ..
Paul Bunnell         1
Wajahat Rauf         1
Jim Issa             1
Kevin Hamedani       1
Tom Brady            1
Name: director_name, Length: 2398, dtype: int64

In [142]:
movies.info() #gives us a summary/description of the data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      5024 non-null   object 
 1   director_name              4939 non-null   object 
 2   num_critic_for_reviews     4993 non-null   float64
 3   duration                   5028 non-null   float64
 4   director_facebook_likes    4939 non-null   float64
 5   actor_3_facebook_likes     5020 non-null   float64
 6   actor_2_name               5030 non-null   object 
 7   actor_1_facebook_likes     5036 non-null   float64
 8   gross                      4159 non-null   float64
 9   genres                     5043 non-null   object 
 10  actor_1_name               5036 non-null   object 
 11  movie_title                5043 non-null   object 
 12  num_voted_users            5043 non-null   int64  
 13  cast_total_facebook_likes  5043 non-null   int64

<h2>Imp Predictors:<br></h2>
genre <br>
director_name<br>
duration<br>
actor_2_name<br>
actor_1_name<br>
movie_title<br>
actor_3_name<br>
plot_keywords<br>
content_rating<br>

In [143]:
movies= movies[['movie_title','genres','director_name','actor_2_name','actor_1_name','actor_3_name','plot_keywords','content_rating']] #keeping only the predictor columns 
movies

,movie_title,genres,director_name,actor_2_name,actor_1_name,actor_3_name,plot_keywords,content_rating
0,Avatar,Action|Adventure|Fantasy|Sci-Fi,James Cameron,Joel David Moore,CCH Pounder,Wes Studi,avatar|future|marine|native|paraplegic,PG-13
1,Pirates of the Caribbean: At World's End,Action|Adventure|Fantasy,Gore Verbinski,Orlando Bloom,Johnny Depp,Jack Davenport,goddess|marriage ceremony|marriage proposal|pi...,PG-13
2,Spectre,Action|Adventure|Thriller,Sam Mendes,Rory Kinnear,Christoph Waltz,Stephanie Sigman,bomb|espionage|sequel|spy|terrorist,PG-13
3,The Dark Knight Rises,Action|Thriller,Christopher Nolan,Christian Bale,Tom Hardy,Joseph Gordon-Levitt,deception|imprisonment|lawlessness|police offi...,PG-13
4,Star Wars: Episode VII - The Force Awakens ...,Documentary,Doug Walker,Rob Walker,Doug Walker,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
5038,Signed Sealed Delivered,Comedy|Drama,Scott Smith,Daphne Zuniga,Eric Mabius,Crystal Lowe,fraud|postal worker|prison|theft|trial,NaN
5039,The Following,Crime|Drama|Mystery|Thriller,NaN,Valorie Curry,Natalie Zea,Sam Underwood,cult|fbi|hideout|prison escape|serial killer,TV-14
5040,A Plague So Pleasant,Drama|Horror|Thriller,Benjamin Roberds,Maxwell Moody,Eva Boehnke,David Chandler,NaN,NaN
5041,Shanghai Calling,Comedy|Drama|Romance,Daniel Hsia,Daniel Henney,Alan Ruck,Eliza Coupe,NaN,PG-13


In [144]:
movies.isnull().sum() #checking if there are any null values in the data

movie_title         0
genres              0
director_name     104
actor_2_name       13
actor_1_name        7
actor_3_name       23
plot_keywords     153
content_rating    303
dtype: int64

In [145]:
movies.fillna('', inplace= True)  #replaces null values with space

In [146]:
movies.isnull().sum()

movie_title       0
genres            0
director_name     0
actor_2_name      0
actor_1_name      0
actor_3_name      0
plot_keywords     0
content_rating    0
dtype: int64

In [147]:
movies.duplicated(['movie_title']).sum() #Do not want to have duplicate entries for the same movie, so it checks if there are any rows with the same movie name

126

In [148]:
len(movies['movie_title'].unique()) #How many unique movie names exist 

4917

In [149]:
movies.shape

(5043, 8)

In [150]:
#movies_unique = movies_unique.groupby('movie_title').agg({'genres':','.join,'director_name':','.join,'actor_2_name':','.join,'actor_1_name':','.join,'actor_3_name':','.join,'plot_keywords':','.join,'content_rating':','.join})#.reset_index()
movies_unique=movies.drop_duplicates(['movie_title']).reset_index(drop=True)#Dropping the rows that have the duplicate movie names and resetting the index, otherwise, for values greater than 4917 it starts to show an error

In [151]:
movies_unique

,movie_title,genres,director_name,actor_2_name,actor_1_name,actor_3_name,plot_keywords,content_rating
0,Avatar,Action|Adventure|Fantasy|Sci-Fi,James Cameron,Joel David Moore,CCH Pounder,Wes Studi,avatar|future|marine|native|paraplegic,PG-13
1,Pirates of the Caribbean: At World's End,Action|Adventure|Fantasy,Gore Verbinski,Orlando Bloom,Johnny Depp,Jack Davenport,goddess|marriage ceremony|marriage proposal|pi...,PG-13
2,Spectre,Action|Adventure|Thriller,Sam Mendes,Rory Kinnear,Christoph Waltz,Stephanie Sigman,bomb|espionage|sequel|spy|terrorist,PG-13
3,The Dark Knight Rises,Action|Thriller,Christopher Nolan,Christian Bale,Tom Hardy,Joseph Gordon-Levitt,deception|imprisonment|lawlessness|police offi...,PG-13
4,Star Wars: Episode VII - The Force Awakens ...,Documentary,Doug Walker,Rob Walker,Doug Walker,,,
...,...,...,...,...,...,...,...,...
4912,Signed Sealed Delivered,Comedy|Drama,Scott Smith,Daphne Zuniga,Eric Mabius,Crystal Lowe,fraud|postal worker|prison|theft|trial,
4913,The Following,Crime|Drama|Mystery|Thriller,,Valorie Curry,Natalie Zea,Sam Underwood,cult|fbi|hideout|prison escape|serial killer,TV-14
4914,A Plague So Pleasant,Drama|Horror|Thriller,Benjamin Roberds,Maxwell Moody,Eva Boehnke,David Chandler,,
4915,Shanghai Calling,Comedy|Drama|Romance,Daniel Hsia,Daniel Henney,Alan Ruck,Eliza Coupe,,PG-13


In [152]:
movies_unique.duplicated(['movie_title']).sum()

0

In [153]:
#editing the genres and plot keywords columns
#removing the | used to separate genres and replacing it with ','
movies_unique['genres']= movies_unique['genres'].apply(lambda a: str(a).replace('|',','))
movies_unique['plot_keywords']= movies_unique['plot_keywords'].apply(lambda a: str(a).replace('|',','))
movies_unique

,movie_title,genres,director_name,actor_2_name,actor_1_name,actor_3_name,plot_keywords,content_rating
0,Avatar,"Action,Adventure,Fantasy,Sci-Fi",James Cameron,Joel David Moore,CCH Pounder,Wes Studi,"avatar,future,marine,native,paraplegic",PG-13
1,Pirates of the Caribbean: At World's End,"Action,Adventure,Fantasy",Gore Verbinski,Orlando Bloom,Johnny Depp,Jack Davenport,"goddess,marriage ceremony,marriage proposal,pi...",PG-13
2,Spectre,"Action,Adventure,Thriller",Sam Mendes,Rory Kinnear,Christoph Waltz,Stephanie Sigman,"bomb,espionage,sequel,spy,terrorist",PG-13
3,The Dark Knight Rises,"Action,Thriller",Christopher Nolan,Christian Bale,Tom Hardy,Joseph Gordon-Levitt,"deception,imprisonment,lawlessness,police offi...",PG-13
4,Star Wars: Episode VII - The Force Awakens ...,Documentary,Doug Walker,Rob Walker,Doug Walker,,,
...,...,...,...,...,...,...,...,...
4912,Signed Sealed Delivered,"Comedy,Drama",Scott Smith,Daphne Zuniga,Eric Mabius,Crystal Lowe,"fraud,postal worker,prison,theft,trial",
4913,The Following,"Crime,Drama,Mystery,Thriller",,Valorie Curry,Natalie Zea,Sam Underwood,"cult,fbi,hideout,prison escape,serial killer",TV-14
4914,A Plague So Pleasant,"Drama,Horror,Thriller",Benjamin Roberds,Maxwell Moody,Eva Boehnke,David Chandler,,
4915,Shanghai Calling,"Comedy,Drama,Romance",Daniel Hsia,Daniel Henney,Alan Ruck,Eliza Coupe,,PG-13


In [154]:
#Removing spaces between words so that the created tags do not overlap. 
#Eg- Sam Mendes makes 2 tags- Sam and Mendes and similarly Sam Underwood makes two tags- Sam and Underwood. 
#This can be confusing because the two Sam's are actually diff people but the model sees it as one tag- Sam. 
#So we have to transform the data in all columns and make them as SamUnderwood etc.
movies_unique['director_name']= movies_unique['director_name'].apply(lambda a: str(a).replace(' ',''))
movies_unique['actor_2_name']= movies_unique['actor_2_name'].apply(lambda a: str(a).replace(' ',''))
movies_unique['actor_3_name']= movies_unique['actor_3_name'].apply(lambda a: str(a).replace(' ',''))
movies_unique['actor_1_name']= movies_unique['actor_1_name'].apply(lambda a: str(a).replace(' ',''))
movies_unique['movie_title']= movies_unique['movie_title'].apply(lambda a: str(a).replace('\xa0',''))
movies_unique

,movie_title,genres,director_name,actor_2_name,actor_1_name,actor_3_name,plot_keywords,content_rating
0,Avatar,"Action,Adventure,Fantasy,Sci-Fi",JamesCameron,JoelDavidMoore,CCHPounder,WesStudi,"avatar,future,marine,native,paraplegic",PG-13
1,Pirates of the Caribbean: At World's End,"Action,Adventure,Fantasy",GoreVerbinski,OrlandoBloom,JohnnyDepp,JackDavenport,"goddess,marriage ceremony,marriage proposal,pi...",PG-13
2,Spectre,"Action,Adventure,Thriller",SamMendes,RoryKinnear,ChristophWaltz,StephanieSigman,"bomb,espionage,sequel,spy,terrorist",PG-13
3,The Dark Knight Rises,"Action,Thriller",ChristopherNolan,ChristianBale,TomHardy,JosephGordon-Levitt,"deception,imprisonment,lawlessness,police offi...",PG-13
4,Star Wars: Episode VII - The Force Awakens ...,Documentary,DougWalker,RobWalker,DougWalker,,,
...,...,...,...,...,...,...,...,...
4912,Signed Sealed Delivered,"Comedy,Drama",ScottSmith,DaphneZuniga,EricMabius,CrystalLowe,"fraud,postal worker,prison,theft,trial",
4913,The Following,"Crime,Drama,Mystery,Thriller",,ValorieCurry,NatalieZea,SamUnderwood,"cult,fbi,hideout,prison escape,serial killer",TV-14
4914,A Plague So Pleasant,"Drama,Horror,Thriller",BenjaminRoberds,MaxwellMoody,EvaBoehnke,DavidChandler,,
4915,Shanghai Calling,"Comedy,Drama,Romance",DanielHsia,DanielHenney,AlanRuck,ElizaCoupe,,PG-13


In [155]:
#combining all predictors other than the movie_title and genre to get a new column-predictors (values separated by '.')
movies_unique['predictors']= movies_unique['genres']+','+movies_unique['actor_1_name']+',' +movies_unique['actor_2_name']+','+ movies_unique['actor_3_name']+ ','+movies_unique['content_rating']+ ','+movies_unique['director_name']+ ','+movies_unique['plot_keywords']
movies_unique

,movie_title,genres,director_name,actor_2_name,actor_1_name,actor_3_name,plot_keywords,content_rating,predictors
0,Avatar,"Action,Adventure,Fantasy,Sci-Fi",JamesCameron,JoelDavidMoore,CCHPounder,WesStudi,"avatar,future,marine,native,paraplegic",PG-13,"Action,Adventure,Fantasy,Sci-Fi,CCHPounder,Joe..."
1,Pirates of the Caribbean: At World's End,"Action,Adventure,Fantasy",GoreVerbinski,OrlandoBloom,JohnnyDepp,JackDavenport,"goddess,marriage ceremony,marriage proposal,pi...",PG-13,"Action,Adventure,Fantasy,JohnnyDepp,OrlandoBlo..."
2,Spectre,"Action,Adventure,Thriller",SamMendes,RoryKinnear,ChristophWaltz,StephanieSigman,"bomb,espionage,sequel,spy,terrorist",PG-13,"Action,Adventure,Thriller,ChristophWaltz,RoryK..."
3,The Dark Knight Rises,"Action,Thriller",ChristopherNolan,ChristianBale,TomHardy,JosephGordon-Levitt,"deception,imprisonment,lawlessness,police offi...",PG-13,"Action,Thriller,TomHardy,ChristianBale,JosephG..."
4,Star Wars: Episode VII - The Force Awakens ...,Documentary,DougWalker,RobWalker,DougWalker,,,,"Documentary,DougWalker,RobWalker,,,DougWalker,"
...,...,...,...,...,...,...,...,...,...
4912,Signed Sealed Delivered,"Comedy,Drama",ScottSmith,DaphneZuniga,EricMabius,CrystalLowe,"fraud,postal worker,prison,theft,trial",,"Comedy,Drama,EricMabius,DaphneZuniga,CrystalLo..."
4913,The Following,"Crime,Drama,Mystery,Thriller",,ValorieCurry,NatalieZea,SamUnderwood,"cult,fbi,hideout,prison escape,serial killer",TV-14,"Crime,Drama,Mystery,Thriller,NatalieZea,Valori..."
4914,A Plague So Pleasant,"Drama,Horror,Thriller",BenjaminRoberds,MaxwellMoody,EvaBoehnke,DavidChandler,,,"Drama,Horror,Thriller,EvaBoehnke,MaxwellMoody,..."
4915,Shanghai Calling,"Comedy,Drama,Romance",DanielHsia,DanielHenney,AlanRuck,ElizaCoupe,,PG-13,"Comedy,Drama,Romance,AlanRuck,DanielHenney,Eli..."


In [156]:
#creating the final dataframe
movies_final = movies_unique[['movie_title','predictors']]
movies_final

,movie_title,predictors
0,Avatar,"Action,Adventure,Fantasy,Sci-Fi,CCHPounder,Joe..."
1,Pirates of the Caribbean: At World's End,"Action,Adventure,Fantasy,JohnnyDepp,OrlandoBlo..."
2,Spectre,"Action,Adventure,Thriller,ChristophWaltz,RoryK..."
3,The Dark Knight Rises,"Action,Thriller,TomHardy,ChristianBale,JosephG..."
4,Star Wars: Episode VII - The Force Awakens ...,"Documentary,DougWalker,RobWalker,,,DougWalker,"
...,...,...
4912,Signed Sealed Delivered,"Comedy,Drama,EricMabius,DaphneZuniga,CrystalLo..."
4913,The Following,"Crime,Drama,Mystery,Thriller,NatalieZea,Valori..."
4914,A Plague So Pleasant,"Drama,Horror,Thriller,EvaBoehnke,MaxwellMoody,..."
4915,Shanghai Calling,"Comedy,Drama,Romance,AlanRuck,DanielHenney,Eli..."


In [157]:
#converting all predictors to lower case to remove ambiguity dur to diff writing formats of data
movies_final['predictors']= movies_final['predictors'].apply(lambda x: x.lower())
movies_final['movie_title']= movies_final['movie_title'].apply(lambda x: x.lower())
#movies_final['genres']= movies_final['genres'].apply(lambda x: x.lower())
movies_final

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,movie_title,predictors
0,avatar,"action,adventure,fantasy,sci-fi,cchpounder,joe..."
1,pirates of the caribbean: at world's end,"action,adventure,fantasy,johnnydepp,orlandoblo..."
2,spectre,"action,adventure,thriller,christophwaltz,roryk..."
3,the dark knight rises,"action,thriller,tomhardy,christianbale,josephg..."
4,star wars: episode vii - the force awakens ...,"documentary,dougwalker,robwalker,,,dougwalker,"
...,...,...
4912,signed sealed delivered,"comedy,drama,ericmabius,daphnezuniga,crystallo..."
4913,the following,"crime,drama,mystery,thriller,nataliezea,valori..."
4914,a plague so pleasant,"drama,horror,thriller,evaboehnke,maxwellmoody,..."
4915,shanghai calling,"comedy,drama,romance,alanruck,danielhenney,eli..."


In [158]:
movies_final.loc[0,:]

movie_title                                               avatar
predictors     action,adventure,fantasy,sci-fi,cchpounder,joe...
Name: 0, dtype: object

In [159]:
movies_final['predictors'][0]

'action,adventure,fantasy,sci-fi,cchpounder,joeldavidmoore,wesstudi,pg-13,jamescameron,avatar,future,marine,native,paraplegic'

<h2> PART 2: RECOMMENDATION SYSTEM </h2>

In [160]:
cv = CountVectorizer(max_features=5000,stop_words='english') 
#will convert words to vectors using this, so select 5000 most common words
#and don't use stop words from English

In [161]:
vector = cv.fit_transform(movies_final['predictors']).toarray() # we use cv to create the vector
# we use toarray() because cv.fit_transform returns a sparse matrix i.e this matrix has a lot of 0s
# and wee need numpy array to perform cosine similarity so we convert it here
vector

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [162]:
vector.shape

(4917, 5000)

In [163]:
vector[0]

array([0, 0, 1, ..., 0, 0, 0])

In [164]:
#CountVectorizer allows us to see these most common 5000  in the corpus(all tags concatenated together is called corpus).
cv.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['11',
 '12',
 '13',
 '14',
 '16th',
 '17',
 '17th',
 '1800s',
 '1880s',
 '18th',
 '1910s',
 '1920s',
 '1930s',
 '1940s',
 '1950s',
 '1960s',
 '1970s',
 '1980s',
 '1990s',
 '19th',
 '2000s',
 '2001',
 '2010s',
 '21st',
 '23rd',
 '3d',
 '50cent',
 'aaronstanford',
 'aaronyoo',
 'aasifmandvi',
 'abandoned',
 'abbiecornish',
 'abduction',
 'abhishekbachchan',
 'abigailspencer',
 'aborigine',
 'abortion',
 'abrahambenrubi',
 'abrams',
 'absent',
 'abu',
 'abuse',
 'abusive',
 'accent',
 'accident',
 'accidental',
 'acclaimed',
 'accountant',
 'accusation',
 'action',
 'activist',
 'actor',
 'actress',
 'ad',
 'adamarkin',
 'adambaldwin',
 'adambrown',
 'adamgarcia',
 'adamgoldberg',
 'adamlefevre',
 'adammckay',
 'adamrifkin',
 'adamsandler',
 'adamscott',
 'adamshankman',
 'adaptation',
 'addict',
 'addiction',
 'address',
 'adoption',
 'adrianlyne',
 'adrianmartinez',
 'adriennebarbeau',
 'adult',
 'adulterous',
 'adultery',
 'adventure',
 'advertising',
 'advice',
 'affair',
 'affairs',

In [165]:
#affair and affairs considered two diff tags. Similarly, attemp and attempted and so on. So too convert all these to one word-root word, we perform stemming.
!pip install nltk.  #nltk is the nlp lib used for stemming

ERROR: Invalid requirement: 'nltk.'


In [166]:
ps = PorterStemmer()

In [167]:
def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [168]:
movies_final['predictors'][0]


'action,adventure,fantasy,sci-fi,cchpounder,joeldavidmoore,wesstudi,pg-13,jamescameron,avatar,future,marine,native,paraplegic'

In [169]:
stem('affairs')

'affair'

In [170]:
movies_final['predictors'] = movies_final['predictors'].apply(stem)
movies_final

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,movie_title,predictors
0,avatar,"action,adventure,fantasy,sci-fi,cchpounder,joe..."
1,pirates of the caribbean: at world's end,"action,adventure,fantasy,johnnydepp,orlandoblo..."
2,spectre,"action,adventure,thriller,christophwaltz,roryk..."
3,the dark knight rises,"action,thriller,tomhardy,christianbale,josephg..."
4,star wars: episode vii - the force awakens ...,"documentary,dougwalker,robwalker,,,dougwalker,"
...,...,...
4912,signed sealed delivered,"comedy,drama,ericmabius,daphnezuniga,crystallo..."
4913,the following,"crime,drama,mystery,thriller,nataliezea,valori..."
4914,a plague so pleasant,"drama,horror,thriller,evaboehnke,maxwellmoody,..."
4915,shanghai calling,"comedy,drama,romance,alanruck,danielhenney,eli..."


In [171]:
#repeating Vectorization process
vector1 = cv.fit_transform(movies_final['predictors']).toarray()
vector1.shape

(4917, 5000)

In [172]:
vector1[0]

array([0, 0, 0, ..., 0, 0, 0])

In [173]:
cv.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['007',
 '11',
 '12',
 '13',
 '14',
 '16th',
 '17',
 '17th',
 '1800s',
 '1880s',
 '18th',
 '1910s',
 '1920s',
 '1930s',
 '1940s',
 '1950s',
 '1960s',
 '1970s',
 '1980s',
 '1990s',
 '19th',
 '2000s',
 '2001',
 '2010s',
 '21st',
 '23rd',
 '3d',
 '50cent',
 'aaronstanford',
 'aaronyoo',
 'aasifmandvi',
 'abandon',
 'abbiecornish',
 'abduction',
 'abhishekbachchan',
 'abigailspencer',
 'aborigine',
 'abortion',
 'abrahambenrubi',
 'abrams',
 'abs',
 'abu',
 'abus',
 'abuse',
 'accent',
 'accident',
 'acclaimed',
 'accountant',
 'accusation',
 'act',
 'action',
 'activist',
 'actor',
 'actress',
 'ad',
 'adamarkin',
 'adambaldwin',
 'adambrown',
 'adamgarcia',
 'adamgoldberg',
 'adamlefevre',
 'adammckay',
 'adamrifkin',
 'adamsandler',
 'adamscott',
 'adamshankman',
 'adaptation',
 'addict',
 'addiction',
 'address',
 'adopt',
 'adoption',
 'adrianlyne',
 'adrianmartinez',
 'adriennebarbeau',
 'adult',
 'adulter',
 'adultery',
 'adventure',
 'advertising',
 'advice',
 'affair',
 'afghanist

In [174]:
#now all these 4917 movies are vectors in a 4917 X 5000 dimensional space.
#To find similarity we calculate distance between each vector. But we don't find eucledean distance.
#We find cosine distance i.e the angle between the two vectors. The smaller the angle the less the distance between the two vectors.
#we do this because in high dimensional data- Euclidean dist is not a good meausure and it fails. 
#In high dimesnional data- sparsity increases i.e data points become far from each other

In [175]:
similarity = cosine_similarity(vector1)
similarity

array([[1.        , 0.32410186, 0.29649973, ..., 0.        , 0.21821789,
        0.06681531],
       [0.32410186, 1.        , 0.26906912, ..., 0.        , 0.19802951,
        0.06063391],
       [0.29649973, 0.26906912, 1.        , ..., 0.16012815, 0.22645541,
        0.06933752],
       ...,
       [0.        , 0.        , 0.16012815, ..., 1.        , 0.23570226,
        0.        ],
       [0.21821789, 0.19802951, 0.22645541, ..., 0.23570226, 1.        ,
        0.10206207],
       [0.06681531, 0.06063391, 0.06933752, ..., 0.        , 0.10206207,
        1.        ]])

In [176]:
similarity.shape #every movie has a similrity with every other movie

(4917, 4917)

In [177]:
similarity[0] # the diagonal of similarity will be 1 because every movie will have perfect similarity with itself (cos 0=1)

array([1.        , 0.32410186, 0.29649973, ..., 0.        , 0.21821789,
       0.06681531])

In [178]:
movies_final[movies_final['movie_title']== 'avatar'].index[0]

0

In [185]:
def recommend(movie):
  movie_index = movies_final[movies_final['movie_title']== movie.lower()].index[0]
  distances= sorted(list(enumerate(similarity[movie_index])),reverse=True,key = lambda x: x[1])
  for i in distances[1:11]:
    print(movies_final.iloc[i[0]].movie_title)

The enumerate() function assigns an index to each item in an iterable object that can be used to reference the item later.
So, we will enumerate over similairty so that each movie gets an index and after sorting we can find out which index has the most similarity(In similarity we know that thw first term is the correlation with the first movie, second with second and so on.) If we directly sort we will get the highest similarities but we won't know which movie does that number belong to, So, we use enumerate to assign index. and we use key because we don't want to sort in descending order base on the assigned index but on. the basis of the similarity.

In [186]:
recommend('avatar')

serenity
waterworld
mystery men
beastmaster 2: through the portal of time
the omega code
hellboy ii: the golden army
x-men: the last stand
terminator salvation
terminator genisys
star trek into darkness


In [187]:
recommend('signed sealed delivered')

the knife of don juan
closer to the moon
theresa is a mother
i love you phillip morris
mean machine
10,000 b.c.            
the big short
a tale of three cities
aloft
treading water


In [188]:
recommend('The Kids Are All Right')

the switch
delivery man
the knife of don juan
and then came love
closer to the moon
theresa is a mother
american desi
10,000 b.c.            
tidal wave
a tale of three cities
